# Part 2 - Implementation 

In your report, describe the system, motivate the choices you made. Comment on the strength and the weaknesses of the system. Suggest 5 movies for each of the first 5 users, Vincent, Edgar, Addilyn, Marlee and Javier, that they haven't rated already.

In [1]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 84.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=1198865 sha256=98c2904de8c2bc441f9ef436702d24d9523856cc8f9f7a0ab292c6fbe0f39083
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as  pd
import numpy as np

from surprise import SVD, accuracy
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate
from surprise.model_selection.split import train_test_split as TTS1

from sklearn.model_selection import train_test_split as TTS2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [3]:
mov_df = pd.read_csv('movie_genres.csv').drop("Unnamed: 0",axis=1)
mov_df['movie_id']= pd.factorize(mov_df['movie_title'])[0]

rev_df = pd.read_csv('user_reviews.csv')
rev_df['user_id']= pd.factorize(rev_df['User'])[0]

df_melt = rev_df.melt(id_vars=['user_id'], value_vars=rev_df.columns.difference(['User','user_id']))
df_melt.rename(columns = {'variable':'movie_title'}, inplace=True)

df_merge = pd.merge(df_melt, mov_df, on='movie_title').drop('movie_title',axis=1)
df_merge.insert(1, 'movie_id', df_merge.pop('movie_id'))

In [4]:
reader = Reader(line_format = 'user item rating', rating_scale=(0,5))
SVD_data = Dataset.load_from_df(df_merge[['user_id','movie_id', 'value']],reader)

train, test = TTS1(SVD_data, test_size=.1, random_state=42)

# initial model
algo = SVD(random_state = 4)
algo.fit(train)
pred = algo.test(test)

# evaluate the rmse result of the prediction and ground thuth
accuracy.rmse(pred)

RMSE: 0.4305


0.43049735260277955

### Test: KNN

In [5]:
data_pred = df_merge.loc[df_merge['value']==0]
X_pred = data_pred.drop('value', axis=1)
Y_pred = data_pred['value']

data_rated = df_merge.loc[df_merge['value']>0]

X = data_rated.drop('value', axis=1)
Y = data_rated['value']

Xtrain, Xtest, Ytrain, Ytest = TTS2(X,Y, test_size= .2, random_state=42)

clf = KNeighborsClassifier(n_neighbors=500)
clf.fit(Xtrain,Ytrain)

pred = clf.predict(Xtest)
exact_acc = accuracy_score(Ytest,pred)

bin_rev_real = np.where(Ytest >= 3,1,0)
bin_rev_pred = np.where(pred >=3,1,0)

bin_acc =  sum(a == b for a, b in zip(bin_rev_real, bin_rev_pred)) / len(bin_rev_real)

print(exact_acc)
print(bin_acc)

0.2886535552193646
0.7742813918305598


In [9]:
data_pred


,user_id,movie_id,value,genre_action,genre_adventure,genre_animation,genre_biography,genre_comedy,genre_crime,genre_documentary,...,genre_mystery,genre_news,genre_reality-tv,genre_romance,genre_sci-fi,genre_short,genre_sport,genre_thriller,genre_war,genre_western
0,0,440,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,1,440,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,2,440,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,3,440,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,4,440,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,595,1922,0.0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1199996,596,1922,0.0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1199997,597,1922,0.0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1199998,598,1922,0.0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [6]:
pred

array([3., 3., 3., ..., 5., 5., 3.])

In [10]:
for i in range(5):
    unseen_movies = X_pred.loc[X_pred['user_id']==i].copy().reset_index(drop=True)
    unseen_movies.insert(2,'score', clf.predict(unseen_movies))
    unseen_movies.sort_values(by = ['score'], ascending=False, inplace=True)
    print(pd.DataFrame(unseen_movies[['user_id','movie_id','score']]).head())


      user_id  movie_id  score
1682        0       139    5.0
460         0      1367    5.0
945         0      1326    5.0
1591        0      1190    5.0
946         0       273    5.0
      user_id  movie_id  score
1779        1        43    5.0
1746        1        84    5.0
1238        1       214    5.0
1727        1      1355    5.0
1237        1       178    5.0
      user_id  movie_id  score
1576        2        12    5.0
855         2         6    5.0
219         2       155    5.0
1662        2       229    5.0
857         2        93    5.0
      user_id  movie_id  score
1089        3       137    5.0
1689        3       139    5.0
361         3        80    5.0
359         3       158    5.0
1676        3        39    5.0
      user_id  movie_id  score
1087        4       137    5.0
365         4       247    5.0
362         4         9    5.0
361         4        80    5.0
1267        4        72    5.0


# Part 3 - Discussion 

Assessing the quality of a recommendation system before deploying it to users is difficult. Why? In a few paragraphs, discuss fundamental challenges in the evaluation of recommendation systems and how they may lead to problems in practice.

Kais förslag: Before getting the system online it's very hard to predict if the system will be good at accomplishing the chosen objective. There are many challenges to building successful recommenders, one of which is choosing what to measure and how to link the metrics to the end effect you seek. Recommenders are overall also difficult due to the fact that you must find something that you actually can measure, and that still is a good measure of your objective. Creating long chains of proxies to link the measurable thing and the objective might lead to a lot of loose connections and a hard time telling where the chain failed if it does not work in practice. 

Another challenge to recommenders, that use ratings or reviews, is the usually very low number of item/user pairs of these ratings. Ie. Most users have not rated most items, making the data incredibly sparse. Having a lot of sparse data, that is not random either, will make it harder to generalize. Despite having a lot of data to test with, the real generalization will first show after the launch of the system in practice. Despite there being a couple of different ways to cope with the spare data, it still poses a limitation in the development phase of the system trying to find weak signals among the noise.      

A third challenging aspect is to determine future behavior from historic data. What makes it harder still is predicting complex choices for users that are not necessarily rational and following the historic trends, but rather combinations of trending themes and personal tastes. Complex social trends and individual choices on large scale are intrinsically hard to predict and something that won't show until after getting the system online.

Finally, the recommendation must also be presented at the right point in time and in the right place. A system can use both push and pull approaches to recommendations. While giving the right sort of recommendation to a user, doing so via another mail in an already full inbox might not be the best way to reach the user. Similarly, the content inventory is crucial. Not even the best recommender systems will do any good if the product-to-market fit is poor. 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=430916cd-0b84-4120-a50c-61096e15b16a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>